### Semantic kernel Vector Search with chat context/history 

In previous session, we walked through a simple example based on a fictitious company called Contoso, Ltd. In that example, a employee wanted to simply connect to company's data, retrieve some info, and add a new record.

Now let's consider a different scenario. 

Let's assume the company has a chatbot that provides below basic support to customers:
1. Provide state of shipment
2. Provide state of return
2. Asking about product info and availability
3. Recommend best product based on customer needs

For an optimal experience, the chatbot needs to not only access product inventory, sales, shipment and return data, but also it would need to be able to pick up the conversation with each customer where it was left off if required.

For this usecase, we are going to introduce and use below semantic kernel capabilities (in addition to what we covered before) to develop a solution:
- Chat completion **Agent** to manage the conversation and data access (one agent)
- Store and maintain **chat history** per customer and retrieve the history when needed.
  -- We will persist the history in **CosmosDB**
- Vector search using Semantic Kernel Vector Store connectors
  -- Use company data as context to guide the conversation aka **RAG**



##### We start with importing the necessary libraries

In [ ]:
from semantic_kernel.kernel import Kernel
from semantic_kernel.connectors.ai.open_ai import AzureChatCompletion

from semantic_kernel.kernel import Kernel
from semantic_kernel.connectors.ai.open_ai import AzureChatCompletion

from semantic_kernel.connectors.ai.function_choice_behavior import (
    FunctionChoiceBehavior,
)
from semantic_kernel.contents.chat_history import ChatHistory

from semantic_kernel.connectors.ai.open_ai.prompt_execution_settings.azure_chat_prompt_execution_settings import (
    AzureChatPromptExecutionSettings,
)

from semantic_kernel.connectors.ai.open_ai import AzureChatCompletion
from semantic_kernel.agents import ChatCompletionAgent
from semantic_kernel.agents import ChatHistoryAgentThread


### Expanding the plugin

In previous session, we defined a class that enabled some basic functionalities to interact with the database. Now let's think about what data would the chatbot need to access and create a plugin to pull in that data:
- For this example we design the Chatbot so that it requires the customer to provide a unique identifier, and also a tracking number if they are following up on a previous conversation. 

Chatbot will need to access the following data to be able to assist the customer:
    - All sales data for the customer
    - All Return data for the customer
    - All basic customer info
    - All reviews and ratings provided by that customer
    - Data about all product offerings.

Below we will expand the plugin class to accomodate above requirements:


In [156]:
import psycopg2
from pandas import DataFrame
from typing import Optional
from semantic_kernel.functions import kernel_function


class Contoso_ChatPlugin:
    def __init__(self, db_uri: str):
        self.conn = psycopg2.connect(db_uri)
        self.cursor = self.conn.cursor()
        print("Connected to company's database successfully.")
    
    @kernel_function
    async def get_all_products(self) -> list[dict]:
        """Gets all products info from the database."""
        query = """SELECT * FROM products;"""
        self.cursor.execute(query)
        rows = self.cursor.fetchall()
        columns = [desc[0] for desc in self.cursor.description]
        try:
            products = DataFrame(rows, columns=columns)
            return products.to_dict(orient="records")  # <-- JSON serializable
        except Exception as e:
            print(f"Error fetching products: {e}")
            return None

    @kernel_function
    async def get_product_info(self, product_name: Optional[str] = None, product_id: Optional[int] = None) -> list[dict]:
        """Retrieves product information based on product name or ID."""
        query = """SELECT 
                    product_id,                   
                    name,
                    inventory,
                    price,
                    refurbished,
                    category
                FROM products
                WHERE (LOWER(name) = LOWER(%(product_name)s) AND %(product_name)s IS NOT NULL)
                   OR (product_id = %(product_id)s AND %(product_id)s IS NOT NULL)
                   """
        if not product_name and not product_id:
            print("No valid product name or ID provided.")
            return None
        elif product_id:
            self.cursor.execute(query, {"product_name": None, "product_id": product_id})
        else:
            self.cursor.execute(query, {"product_name": product_name, "product_id": None})

            
        rows = self.cursor.fetchall()
        columns = [desc[0] for desc in self.cursor.description]
        try:
            products= DataFrame(rows, columns=columns)
            products.to_dict(orient="records")  # <-- JSON serializabl
            
            return products.to_dict(orient="records")  # <-- JSON serializabl
        except Exception as e:
            print(f"Error fetching product information: {e}")
            return None
    
    async def get_customer_info(self, given_customer_id: int) -> list[dict]:
        """Gets all product information from the database."""
        query = """SELECT
                        customer_id,
                        city,
                        state,
                        country,
                        sentiment_score,
                        name,
                        email 
                    
                    FROM customers
                WHERE customer_id = given_customer_id;
                   """
        self.cursor.execute(query, {"given_customer_id": given_customer_id})
        row = self.cursor.fetchone()
        if row:
            columns = [desc[0] for desc in self.cursor.description]
            return dict(zip(columns, row))
        else:
            print(f"No record was found for this customer")
            return None
        
    @kernel_function
    def get_customer_sales_info(self, given_customer_id: int) -> Optional[dict]:
        """Returns all sales info for a customer"""
        query = """
            SELECT sales.sales_id, sales.customer_id, sales.quantity,
                sales.product_id, sales.sale_date, products.name, products.price
            FROM sales
            JOIN products ON sales.product_id = products.product_id
            JOIN customers ON sales.customer_id = customers.customer_id
            WHERE customers.customer_id = %(given_customer_id)s;
        """
        self.cursor.execute(query, {"given_customer_id": given_customer_id})
        row = self.cursor.fetchone()
        if row:
            columns = [desc[0] for desc in self.cursor.description]
            return dict(zip(columns, row))
        else:
            print(f"No sales data found for this customer")
            return None
        
    @kernel_function
    def get_customer_returns_info(self, given_customer_id: int) -> Optional[dict]:
        """Returns all returns info for a customer"""
        query = """SELECT 
                        return_items.return_id,
                        return_items.sales_id,
                        return_items.return_status,
                        return_items.reason,
                        return_items.status_date,
                        sales.customer_id,
                        sales.product_id,
                        sales.sale_date
                    FROM 
                        return_items
                    JOIN 
                        sales ON return_items.sales_id = sales.sales_id
                    JOIN
                        customers ON sales.customer_id = customers.customer_id
                    WHERE 
                        customers.customer_id = %(given_customer_id)s;

        """
        self.cursor.execute(query, {"given_customer_id": given_customer_id})
        row = self.cursor.fetchone()
        if row:
            columns = [desc[0] for desc in self.cursor.description]
            return dict(zip(columns, row))
        else:
            print(f"No return data found for this customer")
            return None

    def close_connection(self):
        """Closes the database connection."""
        self.cursor.close()
        self.conn.close()
        print("Database connection closed.")
    



Now, let's define the agent that will manage the conversation with customers. As before, we need to add the chat completion service, as well as required plugins (to interact with postgreSQL data) to a kernel:

In [157]:
from src.get_conn import get_connection_uri

conn_uri = get_connection_uri()
chat_kernel = Kernel()

# Auto-loads defaults from .env file, alternatively you can set endpoint, deployment_name and api_key directly
chat_completion = AzureChatCompletion()
chat_kernel.add_service(chat_completion)


chat_plugin = Contoso_ChatPlugin(db_uri=conn_uri)

chat_kernel.add_plugin(
    plugin = chat_plugin,
    plugin_name = "Chat_Plugin"
) 

Connection uri was rertieved successfully.
Connected to company's database successfully.


KernelPlugin(name='Chat_Plugin', description=None, functions={'get_all_products': KernelFunctionFromMethod(metadata=KernelFunctionMetadata(name='get_all_products', plugin_name='Chat_Plugin', description='Gets all products info from the database.', parameters=[], is_prompt=False, is_asynchronous=True, return_parameter=KernelParameterMetadata(name='return', description='', default_value=None, type_='list[dict]', is_required=True, type_object=<class 'list'>, schema_data={'type': 'array'}, include_in_function_choices=True), additional_properties={}), invocation_duration_histogram=<opentelemetry.metrics._internal.instrument._ProxyHistogram object at 0x000001EBA9416AD0>, streaming_duration_histogram=<opentelemetry.metrics._internal.instrument._ProxyHistogram object at 0x000001EBA9416110>, method=<bound method Contoso_ChatPlugin.get_all_products of <__main__.Contoso_ChatPlugin object at 0x000001EBAA018810>>, stream_method=None), 'get_customer_returns_info': KernelFunctionFromMethod(metadata=K

Next, we will initiate a ChatCompletionAgent, and pass the kernel we just created to it to empower it for executing the tasks. We also add some instructions to define its main responsibilties:

In [158]:
support_agent = ChatCompletionAgent(
    service=AzureChatCompletion(),
    name="SupportAgent",
    kernel=chat_kernel,
    instructions="You are a support agent for Contoso. You can answer questions about products, customers, sales, and returns. Use the ABC_Chat_Plugin to access the company's database."
)


#### Storing and retrieving chat history

As mentioned, chatbot will need to access history of previous converation if needed. Semantic Kernel agents use a class called ChatHistoryAgentThread to store conversation data and keep track of it. So we need to be able to store and retrieve chat history data for each conversation using an isntance of this class.

As mentioned, we choose to store chat history in an Azure Cosmos DB container due to the NoSQL nature of this data. So we need to develop a capability to store the converation history for each customer and chat session to a CosmosDB container.

We have created a database called "Contoso" which has a container called "customer_chats".

First let's connect to the database via the database client, which we can then use to get the container client that we can use to perform read/write operations:

In [148]:
from dotenv import load_dotenv

from azure.cosmos import CosmosClient
import os

load_dotenv()


client = CosmosClient.from_connection_string(os.getenv("COSMOS_CONNECTION_STRING"))

databaseName = os.getenv("COSMOS_DATABASE_NAME")
database = client.get_database_client(databaseName)


containerName =  "customer_chats"
container = database.get_container_client(containerName)

Next, let's define a class to enbale 1- storing chat history data to the container 2- getting the relevant history data from the container and update the ChatHistoryAgentThread object with it:

In [151]:
from semantic_kernel.agents import ChatHistoryAgentThread

from semantic_kernel.contents import ChatMessageContent

class ChatHistoryInCosmosDB(ChatHistoryAgentThread):
    """This class stores the chat history in a Cosmos DB container as plain documents."""

    def __init__(self, session_id: str, customer_id: int, container):
        super().__init__()
        self.session_id = session_id
        self.customer_id = customer_id
        self.container = container

    async def store_history(self):
        """Store the chat history in the Cosmos DB as a document."""
        messages = [msg async for msg in self.get_messages()]
        item = {
            "id": self.session_id,
            "customer_id": str(self.customer_id),
            "messages": [msg.model_dump() for msg in messages],
        }
        try:
            res=self.container.upsert_item(item)
            print(f"Messages stored in Cosmos DB with response: {res}")
        except Exception as e:
            print(f"Error storing messages in Cosmos DB: {e}")
            raise

    async def read_history(self):
        """Read the chat history from the Cosmos DB."""
        try:
            item = self.container.read_item(item=self.session_id, partition_key=str(self.customer_id))
            print(f"Messages read from Cosmos DB")
            for m in item.get("messages", []):    
                # Convert each message to ChatMessageContent
                await self.on_new_message(ChatMessageContent.model_validate(m))
            return True
        except Exception as e:
            print(f"No chat history for this customer and session_id was retrieved: {e}")
            return False

Now let's tie everything together

In [159]:
# customer data
import random

customer_id = 3
session_id =  "session_1624"
user_input = "Hi, my customer ID is " + str(customer_id) + ", session ID is "+ str(session_id)
if(session_id is None):
    session_id = f"session_"+random.randint(1000, 9999).__str__()

# initiate chat history object
hist = ChatHistoryInCosmosDB(
        session_id=session_id,
        customer_id=customer_id,
        container=container
    )
_ = await hist.read_history()


print("Customer >", user_input)
while(True):
    
    response = await support_agent.get_response(messages=user_input,thread=hist)
    print(f"Support Agent: {response}")
    # hist = response.thread

    if user_input.lower() == "exit":
        await hist.store_history()
        print("NOTE: If you would like to continue this chat in the future, use this session ID:", session_id)
        break

    user_input = input("You: ")

    

Messages read from Cosmos DB
Customer > Hi, my customer ID is 3, session ID is session_1624
Support Agent: Welcome back! How can I assist you today with your account (Customer ID: 3)? If you need information about your purchases, returns, or any product details, just let me know!
Support Agent: You have made 1 purchase as a Contoso customer. Here are the details:

- Product: Macbook Pro M4
- Quantity: 3 units
- Purchase date: June 5, 2025

If you have any other questions or need more details about your purchases, please let me know!
Support Agent: Your last question was: "how many times I have purchased an item?"

If you need details about that or have further questions, feel free to ask!
Support Agent: I don’t have a record of recommending a phone to you in our previous conversations. If you’re interested in phone recommendations or want more information about specific models, please let me know your preferences, and I’d be happy to assist!
Support Agent: Thank you for contacting Cont